In [1]:
from learntodrive.models.CNN_Model import densenet121
from learntodrive.dataloader_aug import Drive360Loader

from learntodrive.utils import move_target_to_cuda
from learntodrive.utils import move_data_to_cuda
from learntodrive.utils import log_textfile

import time
import numpy as np
import pandas as pd

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [2]:
def run_validation(model, validation_loader):
  model.eval()
  target_speed = np.array([])
  target_steer = np.array([])
  pred_speed = np.array([])
  pred_steer = np.array([])
  with torch.no_grad():
      for batch_idx, (data, target, front_name) in enumerate(validation_loader):
          data = move_data_to_cuda(data)
          target = move_target_to_cuda(target)
          #resnet34_features = get_features(front_name, resnet34).cuda()
          prediction = model(data, [])
          # Again only evaluating the canSpeed 
          # predictions, add canSteering when 
          # jointly training.
          cur_pred_speed = np.asarray(prediction['canSpeed'].detach().cpu())
          cur_pred_steer = np.asarray(prediction['canSteering'].detach().cpu())
          cur_target_speed = np.asarray(target['canSpeed'].detach().cpu())
          cur_target_steer = np.asarray(target['canSteering'].detach().cpu())
          cur_pred_steer = (cur_pred_steer*target_std['canSteering'])+target_mean['canSteering']
          cur_target_steer = (cur_target_steer*target_std['canSteering'])+target_mean['canSteering']
          cur_pred_speed = (cur_pred_speed*target_std['canSpeed'])+target_mean['canSpeed']
          cur_target_speed = (cur_target_speed*target_std['canSpeed'])+target_mean['canSpeed']
          pred_speed = np.concatenate([pred_speed, cur_pred_speed])
          pred_steer = np.concatenate([pred_steer, cur_pred_steer])
          target_speed = np.concatenate([target_speed, cur_target_speed])
          target_steer = np.concatenate([target_steer, cur_target_steer])

          if (batch_idx+1) % 10 == 0:
                  print("Validation batch: " + str(batch_idx+1))
  mse_steer = (np.square(pred_steer - target_steer)).mean()
  mse_speed = (np.square(pred_speed - target_speed)).mean()
  return(mse_steer, mse_speed)

In [3]:
road_idx = {
    6: 1, #ground
    7: 2, #road
    8: 3, #sidewalk
    9: 4, #parking
    10: 5,#railroad
    11: 6,#building
    12: 6,#wall
    13: 6,#fence
    23: 7,#sky
    24: 8,#person
    25: 9,#rider
    26: 10,#car
    27: 10,#truck
    28: 10,#bus
    29: 10,#caravan
    30: 10,#trailer
    31: 11,#train
    32: 12,#mortocyle
    33: 13 #bycicle   
}

epochs = 100
logfile = './logs/CNN_singleimage.log'

In [4]:
import json
import sys

config = json.load(open('config_sample1.json'))

In [5]:
config['data_loader']['historic']['number'] = 1
config['data_loader']['train']['batch_size'] = 13
config['data_loader']['validation']['batch_size'] = 13
config['data_loader']['test']['batch_size'] = 13

target_mean = config['target']['mean']
target_std = config['target']['std']

In [6]:
# create a train, validation and test data loader
train_loader = Drive360Loader(config, 'train', 0.8, road_idx)
validation_loader = Drive360Loader(config, 'validation', 0, road_idx)
test_loader = Drive360Loader(config, 'test', 0, road_idx)

# print the data (keys) available for use. See full 
# description of each data type in the documents.
print('Loaded train loader with the following data available as a dict.')
print(train_loader.drive360.dataframe.keys())

Phase: train # of data: 67434
Phase: validation # of data: 10590
Phase: test # of data: 28802
Loaded train loader with the following data available as a dict.
Index(['cameraRight', 'cameraFront', 'cameraRear', 'cameraLeft', 'canSteering',
       'canSpeed', 'chapter'],
      dtype='object')


In [7]:
model = densenet121().cuda()

In [8]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003)
lr_decay_idx = 0
lrates = [0.0001, 0.00005, 0.00003, 0.0001]
lowestSpeedLoss = 999999
lowestSteerLoss = 999999
log_textfile(logfile, 'start')
for epoch in range(epochs):
    total_loss = 0.0
    running_loss = 0.0
    end = time.time()
    model.train()
    target_speed = np.array([])
    target_steer = np.array([])
    pred_speed = np.array([])
    pred_steer = np.array([])
    for batch_idx, (data, target, front_name) in enumerate(train_loader):
        data = move_data_to_cuda(data)
        target = move_target_to_cuda(target)
        start = time.time()
        delta_load = start-end
        optimizer.zero_grad()
        prediction = model(data, [])
  
        # Ony optimizing for canSpeed at the moment
        # add canSteering to optimize simulatenously.
        loss_speed = criterion(prediction['canSpeed'], target['canSpeed'])
        loss_steering = criterion(prediction['canSteering'], target['canSteering'])
        loss = loss_steering+loss_speed
        loss.backward()
        optimizer.step()

        cur_pred_speed = np.asarray(prediction['canSpeed'].detach().cpu())
        cur_pred_steer = np.asarray(prediction['canSteering'].detach().cpu())
        cur_target_speed = np.asarray(target['canSpeed'].detach().cpu())
        cur_target_steer = np.asarray(target['canSteering'].detach().cpu())
        cur_pred_steer = (cur_pred_steer*target_std['canSteering'])+target_mean['canSteering']
        cur_target_steer = (cur_target_steer*target_std['canSteering'])+target_mean['canSteering']
        cur_pred_speed = (cur_pred_speed*target_std['canSpeed'])+target_mean['canSpeed']
        cur_target_speed = (cur_target_speed*target_std['canSpeed'])+target_mean['canSpeed']
        pred_speed = np.concatenate([pred_speed, cur_pred_speed])
        pred_steer = np.concatenate([pred_steer, cur_pred_steer])
        target_speed = np.concatenate([target_speed, cur_target_speed])
        target_steer = np.concatenate([target_steer, cur_target_steer])
        
        # print statistics
        running_loss += loss.item()
        total_loss += loss.item()
        end = time.time()
        delta_model = end - start
        if (batch_idx+1) % 100 == 0:  
            log_textfile(logfile, '[epoch: %d, batch:  %5d] loss: %.5f time load: %.5f time model: %.5f' % (epoch + 1, batch_idx + 1, running_loss / 100.0, delta_load, delta_model))
            running_loss = 0.0
    if epoch in [5, 15, 20, 200]:
        for param_group in optimizer.param_groups:
            param_group["lr"] = lrates[lr_decay_idx]
        print('LR:' + str(lrates[lr_decay_idx]))
        lr_decay_idx += 1
        
        
    shuffle(train_loader.drive360.indices)
    mse_steer, mse_speed = run_validation(model, validation_loader)
    mse_steer_t = (np.square(pred_steer - target_steer)).mean()
    mse_speed_t = (np.square(pred_speed - target_speed)).mean()
    torch.save(model, './models/CNN_singleimage_last.pth')
    if mse_steer < lowestSteerLoss:
        torch.save(model, './models/CNN_singleimage_beststeer.pth')
        lowestSteerLoss = mse_steer
    if mse_speed < lowestSpeedLoss:
        torch.save(model, './models/CNN_singleimage_bestspeed.pth')
        lowestSpeedLoss = mse_speed
    log_textfile(logfile, "Epoch: " + str(epoch + 1) + "/" + str(epochs) + " Loss: " + str(total_loss/batch_idx) + " Train MSESteer: " + str(round(mse_steer_t,3)) + " Train MSESpeed: " + str(round(mse_speed_t,3)) + " MSESteer: " + str(round(mse_steer,3)) + " MSESpeed: " + str(round(mse_speed,3)))

print('Best Steer Loss:', lowestSteerLoss)
print('Best Speed Loss:', lowestSpeedLoss)

start
[epoch: 1, batch:    100] loss: 1.54557 time load: 0.00750 time model: 0.21099
[epoch: 1, batch:    200] loss: 1.18142 time load: 0.00791 time model: 0.21021


KeyboardInterrupt: 

In [ ]:
# read in model
modelSteer = torch.load('./models/CNN_singleimage_last.pth')
modelSpeed = torch.load('./models/CNN_singleimage_last.pth')

In [ ]:
normalize_targets = config['target']['normalize']
target_mean = config['target']['mean']
target_std = config['target']['std']

def add_results(results, predictionSteer, predictionSpeed, front_name):
    steering = np.squeeze(predictionSteer['canSteering'].cpu().data.numpy())
    speed = np.squeeze(predictionSpeed['canSpeed'].cpu().data.numpy())
    last_image_names = front_name[0]
    front_name = [x for x in last_image_names]
    image_front_name = np.squeeze(np.array(front_name))
    if normalize_targets:
        steering = (steering*target_std['canSteering'])+target_mean['canSteering']
        speed = (speed*target_std['canSpeed'])+target_mean['canSpeed']
        front_name = front_name
    if np.isscalar(steering):
        steering = [steering]
    if np.isscalar(speed):
        speed = [speed]
    results['canSteering'].extend(steering)
    results['canSpeed'].extend(speed)
    results['cameraFront'].extend(front_name)

In [ ]:
results = {'canSteering': [],
           'canSpeed': [], 
           'cameraFront': []}

modelSteer.eval()
modelSpeed.eval()
with torch.no_grad():
    for batch_idx, (data, target, front_name) in enumerate(validation_loader):
        data = move_data_to_cuda(data)
        target = move_target_to_cuda(target)
        predictionSteer = modelSteer(data, [])
        predictionSpeed = modelSpeed(data, [])
        add_results(results, predictionSteer, predictionSpeed, front_name)
        
df = pd.DataFrame.from_dict(results)

In [ ]:
df['img_idx'] = df['cameraFront'].str[-9:].str[:5].astype(int)
df['canSpeed_1'] = df['canSpeed'].shift(1)
df['canSpeed_2'] = df['canSpeed'].shift(2)
df['canSteering_1'] = df['canSteering'].shift(1)
df['canSteering_2'] = df['canSteering'].shift(2)
df['img_idx_1'] = df['img_idx'].shift(1)
df['img_idx_2'] = df['img_idx'].shift(2)    

## Read in full test data set and merge predictions and interpolate missing values

In [11]:
test_full = pd.read_csv('./Data/test_full.csv')

In [ ]:
# merge the predictions and the original test set
complete_test_set = pd.merge(test_full, df, how='left', on='cameraFront')

index_list = complete_test_set.groupby('chapter').apply(lambda x: x.iloc[100:]).index.droplevel(level=0).tolist()
complete_test_set = complete_test_set.loc[index_list,]

complete_test_set['flag'] = complete_test_set['flag'].fillna(0)

complete_test_set['canSpeed_1'] = complete_test_set.groupby(['chapter'])['canSpeed_1'].bfill(limit=10)
complete_test_set['canSpeed_2'] = complete_test_set.groupby(['chapter'])['canSpeed_2'].bfill(limit=10)
complete_test_set['canSteering_1'] = complete_test_set.groupby(['chapter'])['canSteering_1'].bfill(limit=10)
complete_test_set['canSteering_2'] = complete_test_set.groupby(['chapter'])['canSteering_2'].bfill(limit=10)
complete_test_set['img_idx_1'] = complete_test_set.groupby(['chapter'])['img_idx_1'].bfill(limit=10)
complete_test_set['img_idx_2'] = complete_test_set.groupby(['chapter'])['img_idx_2'].bfill(limit=10)

complete_test_set['extrapolatedStreering'] = (complete_test_set['canSteering_2'] complete_test_set['canSteering_1'])/(complete_test_set['img_idx_2']-complete_test_set['img_idx_1']) * (complete_test_set['img_idx']-complete_test_set['img_idx_1']) + complete_test_set['canSteering_1']
complete_test_set['extrapolatedSpeed'] = (complete_test_set['canSpeed_2']-complete_test_set['canSpeed_1'])/(complete_test_set['img_idx_2']-complete_test_set['img_idx_1']) * (complete_test_set['img_idx']-complete_test_set['img_idx_1']) + complete_test_set['canSpeed_1']

complete_test_set.loc[complete_test_set['canSpeed'].isna(),'canSpeed'] = complete_test_set.loc[complete_test_set['canSpeed'].isna(),'extrapolatedSpeed']
complete_test_set.loc[complete_test_set['canSteering'].isna(),'canSteering'] = complete_test_set.loc[complete_test_set['canSteering'].isna(),'extrapolatedStreering']

complete_test_set['canSpeed'] = complete_test_set['canSpeed'].fillna(13.426163367846936)
complete_test_set['canSteering'] = complete_test_set['canSteering'].fillna(-5.406788214535221)

if np.sum(np.sum(complete_test_set.isna())>0)>0:
    print('Error some NA values!')
    complete_test_set.to_csv('./submissions/error_submission.csv', index=False)
else:
    if complete_test_set.shape[0] != 279863 and False:
        print('Sumbission file has wrong number of lines')
    else:
        print('Submission file has no NAs!')
        submission_cols = complete_test_set[['canSteering', 'canSpeed']]
        submission_cols.to_csv('./submissions/CNN_singleimage_submission.csv', index=False) 